<a href="https://colab.research.google.com/github/dimiporf/datavis/blob/main/100567030_dash_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q plotly==5.4 dash jupyter-dash

In [2]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

WORLD BANK DATA

In [3]:
# Load the Gapminder dataset
wb_data = px.data.gapminder()
wb_data.head()

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
0,Afghanistan,Asia,1952,28.801,8425333,779.445314,AFG,4
1,Afghanistan,Asia,1957,30.332,9240934,820.853030,AFG,4
2,Afghanistan,Asia,1962,31.997,10267083,853.100710,AFG,4
3,Afghanistan,Asia,1967,34.020,11537966,836.197138,AFG,4
4,Afghanistan,Asia,1972,36.088,13079460,739.981106,AFG,4


In [4]:
# Create the app
app = JupyterDash(__name__)

In [5]:
scatter_header = html.H4('GDP and life expectancy evolution by region')

In [6]:
# Define the callback function
@app.callback(
    Output('scatter-graph', 'figure'),
    [Input('scatter-dropdown', 'value')]
)
def select_region(region):
    if region:
        subset = wb_data.query(f"continent == '{region}'")
    else:
        subset = wb_data

    fig = px.scatter(
        subset,
        x='gdpPercap',
        y='lifeExp',
        size='pop',
        color='continent',
        animation_frame='year',
        hover_name='country',
        range_y=[20, 90],
        size_max=60,
        log_x=True
    )

    return fig

In [7]:
# Define the region options for the dropdown
region_options = [
    {'label': 'Worldwide', 'value': ''},
    {'label': 'Asia', 'value': 'Asia'},
    {'label': 'Americas', 'value': 'Americas'},
    {'label': 'Europe', 'value': 'Europe'},
    {'label': 'Africa', 'value': 'Africa'},
    {'label': 'Oceania', 'value': 'Oceania'}
]

In [8]:
# Define the scatter_dropdown component
scatter_dropdown = dcc.Dropdown(
    id='scatter-dropdown',
    options=region_options,
    value=None
)

In [9]:
# Define the scatter_graph component
scatter_graph = dcc.Graph(id='scatter-graph')

In [10]:
scatter_row = html.Div(
    children=[scatter_header,
              scatter_dropdown,
              scatter_graph]
)

CHOROPLETH MAP SECTION

In [11]:
# Define the map_header component
map_header = html.H4('Evolution by Country')

In [12]:
# Define the select_metric callback function
@app.callback(
    Output("map-graph", "figure"),
    [Input("map-radio", "value")]
)
def select_metric(metric):
    fig = px.choropleth(
        wb_data,
        locations='iso_alpha',
        color=metric,
        hover_name='country',
        animation_frame='year'
    )
    return fig

In [13]:
# Define the map_radio component
map_radio = dcc.RadioItems(
    id="map-radio",
    options=[
        {'label': 'GDP per capita', 'value': 'gdpPercap'},
        {'label': 'Life expectancy', 'value': 'lifeExp'},
        {'label': 'Population', 'value': 'pop'}
    ],
    value='gdpPercap'
)

In [14]:
# Define the map_graph component
map_graph = dcc.Graph(id='map-graph')

In [15]:
# Define the map_row component
map_row = html.Div(
    children=[map_header,
              map_radio,
              map_graph]
)

TRENDS CHART

In [16]:
trends_header = html.H4('Evolution of trends by country')

In [17]:
# Define the trends callback function
@app.callback(
    Output("trends-graph", "figure"),
    [Input("trends-radio", "value")]
)
def select_trend_chart(chart):
    if chart is None or chart == 'line':
        fig = px.line(wb_data, x='year', y='lifeExp', color='continent', line_group='country', hover_name='country', line_shape='linear')
    else:
        fig = px.area(wb_data, x='year', y='lifeExp', color='continent', line_group='country', hover_name='country')
    
    return fig

In [18]:
trends_radio= dcc.RadioItems(
    id='trends-radio',
    options=[
        {'label':'Line','value':'line'},
        {'label':'Area','value':'area'},
    ], 
    value='line'
)

In [19]:
trends_graph = dcc.Graph(id="trends-graph")

In [20]:
trends_row = html.Div(
    id='trends-row',
    children=[
        trends_header,
        trends_radio,
        trends_graph
    ]
)

DISTRIBUTION OF PEOPLE BY COUNTRY

In [21]:
distribution_header = html.H4('Population of people in each country')

In [22]:
@app.callback(Output('distribution-graph', 'figure'), [Input('distribution-radio', 'value')])
def select_distribution_chart(chart):
  if chart == 'sunburst':
    fig = px.sunburst(wb_data, path=['continent','country'], values='pop', color='continent', hover_data=['iso_alpha'])
  elif chart == 'treemap':
    fig = px.treemap(wb_data, path=['continent','country'], values='pop', color='continent', hover_data=['iso_alpha'])
  else:
    fig = px.icicle(wb_data, path=['continent','country'], values='pop', color='continent', hover_data=['iso_alpha'])
  return fig



In [23]:
distribution_radio = dcc.RadioItems(
    id='distribution-radio',
    options=[
        {'label':'Sunburst','value':'sunburst'},
        {'label':'Tree Map','value':'treemap'},
        {'label':'Icicle','value':'icile'}
            ],
            value='sunburst'
)

In [24]:
distribution_graph = dcc.Graph(id='distribution-graph')

In [25]:
distribution_row = html.Div(
    id='distribution-row',
    children=[
        distribution_header,
        distribution_radio,
        distribution_graph
    ]
)

CREATE DASHBOARD

In [26]:
# Define the layout
app.layout = html.Div([
    html.H1('Hello World Bank'),
    html.Div(
        children=[
            html.H2('Select a Region:'),
            scatter_dropdown,
            scatter_graph,
            map_row,
            trends_row,
            distribution_row
        ]
    )
])

In [27]:
# Run the app
if __name__ == "__main__":
    app.run_server(mode="inline")


Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>